In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Load Excel files
trending_df = pd.read_excel("trending_topics.xlsx")
product_rate_df = pd.read_excel("product_rate_list.xlsx")
wholesale_price_df = pd.read_excel("wholesale_price.xlsx")

# Filter trending products with popularity score >= 80
trending_filtered = trending_df[trending_df['Popularity Score'] >= 80].copy()
trending_filtered.dropna(subset=['Product / Style'], inplace=True)
trending_products = trending_filtered['Product / Style'].unique()

# Output list
output = []

for product in trending_products:
    # Get competitor prices
    competitors = product_rate_df[product_rate_df['Product Name'] == product]
    if competitors.empty:
        continue
    comp_prices = competitors['Price (INR)'].values
    min_comp_price = comp_prices.min()
    
    # Get wholesale prices
    wholesalers = wholesale_price_df[wholesale_price_df['Product Name'] == product]
    if wholesalers.empty:
        continue
    cheapest_wholesaler = wholesalers.loc[wholesalers['Price (INR)'].idxmin()]
    wholesale_price = cheapest_wholesaler['Price (INR)']
    wholesaler_name = cheapest_wholesaler['Seller']
    
    # Generate training data for regression
    X_train = []
    y_train = []
    for margin in np.arange(0.30, 0.71, 0.01):  # 30% to 70%
        price = wholesale_price * (1 + margin)
        if price < min_comp_price:
            X_train.append([wholesale_price])
            y_train.append(price)
    
    if not X_train:
        continue
    
    # Fit regression model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Predict best price
    predicted_price = model.predict([[wholesale_price]])[0]
    profit_percent = ((predicted_price - wholesale_price) / wholesale_price) * 100

    output.append({
        "Product": product,
        "Wholesaler": wholesaler_name,
        "Wholesale Price (INR)": wholesale_price,
        "Recommended Selling Price (INR)": round(predicted_price, 2),
        "Profit (%)": round(profit_percent, 2),
        "Lowest Competitor Price (INR)": min_comp_price
    })

# Save to Excel
result_df = pd.DataFrame(output)
result_df.to_excel("ml_predicted_prices.xlsx", index=False)
print("File 'ml_predicted_prices.xlsx' created successfully.")


File 'ml_predicted_prices.xlsx' created successfully.
